In [1]:
import pandas as pd

In [2]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_03.csv')

In [3]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [4]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [5]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [6]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [7]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [8]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,WAS,6/3/2021,1,607208,Trea Turner,1,Y,SS
1,WAS,6/3/2021,1,543281,Josh Harrison,2,Y,LF
2,WAS,6/3/2021,1,665742,Juan Soto,3,Y,RF
3,WAS,6/3/2021,1,475582,Ryan Zimmerman,4,Y,1B
4,WAS,6/3/2021,1,516770,Starlin Castro,5,Y,3B
...,...,...,...,...,...,...,...,...
221,SD,6/3/2021,1,543333,Eric Hosmer,5,N,NaN
222,SD,6/3/2021,1,571976,Wil Myers,6,N,NaN
223,SD,6/3/2021,1,595777,Jurickson Profar,7,N,NaN
224,SD,6/3/2021,1,605170,Victor Caratini,8,N,NaN


In [9]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [10]:
pitch_df.drop(456, inplace = True)

In [11]:
matchups

{'ATL': 'WAS',
 'WAS': 'ATL',
 'NYY': 'TB',
 'TB': 'NYY',
 'HOU': 'BOS',
 'BOS': 'HOU',
 'COL': 'TEX',
 'TEX': 'COL',
 'PIT': 'MIA',
 'MIA': 'PIT',
 'MIL': 'ARI',
 'ARI': 'MIL',
 'KC': 'MIN',
 'MIN': 'KC',
 'CHW': 'DET',
 'DET': 'CHW',
 'STL': 'CIN',
 'CIN': 'STL',
 'LAA': 'SEA',
 'SEA': 'LAA',
 'SF': 'CHC',
 'CHC': 'SF',
 'SD': 'NYM',
 'NYM': 'SD'}

In [12]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [13]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    if len(lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values) > 0: 
        opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                                & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
        if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
                opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
                opp_pitcher.append(opp_pitch)
                keep_pitch.append(opp_pitch)
        else:
            del_indexes.append(i)
    else:
        del_indexes.append(i)
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-13-a4b56b56add5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-13-a4b56b56add5>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [14]:
del_indexes

[]

In [15]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-15-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [16]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,WAS,6/3/2021,1,607208,Trea Turner,1,Y,SS,L,Tucker Davidson,...,0.389474,0.453704,0.726316,1.180019,0.336842,0.405405,28.680108,15.516181,0.489726,210.188251
1,WAS,6/3/2021,1,543281,Josh Harrison,2,Y,LF,L,Tucker Davidson,...,0.310345,0.349206,0.551724,0.900930,0.241379,0.291667,11.030115,3.336265,0.381467,138.706304
2,WAS,6/3/2021,1,665742,Juan Soto,3,Y,RF,L,Tucker Davidson,...,0.295455,0.386139,0.625000,1.011139,0.329545,0.283582,20.663026,8.386733,0.414917,160.515070
3,WAS,6/3/2021,1,475582,Ryan Zimmerman,4,Y,1B,L,Tucker Davidson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WAS,6/3/2021,1,516770,Starlin Castro,5,Y,3B,L,Tucker Davidson,...,0.377049,0.400000,0.475410,0.875410,0.098361,0.479167,11.303304,3.572839,0.377694,137.992844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,SD,6/3/2021,1,665487,Fernando Tatis,4,N,NaN,R,Taijuan Walker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,SD,6/3/2021,1,543333,Eric Hosmer,5,N,NaN,R,Taijuan Walker,...,0.293578,0.351464,0.463303,0.814767,0.169725,0.315789,36.246700,7.460574,0.350965,124.187731
222,SD,6/3/2021,1,571976,Wil Myers,6,N,NaN,R,Taijuan Walker,...,0.270742,0.341176,0.502183,0.843360,0.231441,0.326797,40.409832,9.356135,0.359251,129.102742
223,SD,6/3/2021,1,595777,Jurickson Profar,7,N,NaN,R,Taijuan Walker,...,0.265217,0.354717,0.378261,0.732978,0.113043,0.294737,34.644456,2.361405,0.325018,106.918620


In [17]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [18]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [19]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [20]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,WAS,6/3/2021,1,607208,Trea Turner,1,Y,SS,L,Tucker Davidson,...,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143
1,WAS,6/3/2021,1,543281,Josh Harrison,2,Y,LF,L,Tucker Davidson,...,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143
2,WAS,6/3/2021,1,665742,Juan Soto,3,Y,RF,L,Tucker Davidson,...,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
3,WAS,6/3/2021,1,475582,Ryan Zimmerman,4,Y,1B,L,Tucker Davidson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WAS,6/3/2021,1,516770,Starlin Castro,5,Y,3B,L,Tucker Davidson,...,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143,3.857143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,SD,6/3/2021,1,665487,Fernando Tatis,4,N,NaN,R,Taijuan Walker,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,SD,6/3/2021,1,543333,Eric Hosmer,5,N,NaN,R,Taijuan Walker,...,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916
222,SD,6/3/2021,1,571976,Wil Myers,6,N,NaN,R,Taijuan Walker,...,7.774869,7.774869,7.774869,7.774869,7.774869,7.774869,7.774869,7.774869,7.774869,7.774869
223,SD,6/3/2021,1,595777,Jurickson Profar,7,N,NaN,R,Taijuan Walker,...,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916,9.480916


In [21]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [22]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [23]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
8,WAS,6/3/2021,1,571578,Patrick Corbin,SP,Y,NaN,1,7,...,13.216783,1.244804,0.533487,0.711316,2.226463,11.601399,2.441860,5.143939,37.655046,31.707622
17,ATL,6/3/2021,1,656353,Tucker Davidson,SP,Y,NaN,2,6,...,11.571429,0.571429,0.857143,-0.285714,1.371429,9.000000,1.200000,2.678571,41.991407,40.334653
27,TB,6/3/2021,1,642232,Ryan Yarbrough,SP,Y,NaN,2,7,...,8.560976,1.289003,0.268542,1.020460,1.740331,7.682927,1.911348,4.676409,29.226077,29.602864
37,NYY,6/3/2021,1,543037,Gerrit Cole,SP,Y,NaN,3,6,...,4.666667,1.910828,0.286624,1.624204,1.228188,5.629630,1.687500,4.803625,14.256227,17.053693
47,BOS,6/3/2021,1,527048,Martin Perez,SP,Y,NaN,1,8,...,7.397260,1.545894,0.753623,0.792270,1.983651,10.684932,2.306050,6.266667,33.464783,38.338110
57,HOU,6/3/2021,1,543606,Jake Odorizzi,SP,Y,NaN,2,7,...,21.807692,1.657895,0.276316,1.381579,1.971831,9.288462,2.041667,3.141026,43.930198,30.234004
66,TEX,6/3/2021,1,592314,Mike Foltynewicz,SP,Y,NaN,3,5,...,7.620968,0.946746,0.266272,0.680473,1.329114,6.169355,1.458333,3.436019,22.589570,21.549870
75,COL,6/3/2021,1,592351,Jon Gray,SP,Y,NaN,2,6,...,5.142857,1.240223,0.469274,0.770950,1.527950,7.857143,1.804878,4.285714,22.223998,24.793281
84,MIA,6/3/2021,1,622694,Elieser Hernandez,SP,N,NaN,4,4,...,9.000000,1.272727,0.181818,1.090909,0.774194,3.750000,0.864865,3.461538,18.220812,14.197375
93,PIT,6/3/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,7.280899,1.218997,0.538259,0.680739,1.495496,7.887640,1.670588,4.387352,26.192616,30.090019


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [24]:
scores = pd.read_csv('../datasets/6_03_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210603,145u,642232,"Yarbrough, Ryan",1.0,0,1,31.15,7300,TAM,@ nyy,NaN,9,2
1,20210603,146x,596295,"Gomber, Austin",1.0,9,1,27.10,5400,COL,v tex,NaN,11,6
2,20210603,101g,527048,"Perez, Martin",1.0,0,1,25.05,7600,BOS,@ hou,NaN,5,1
3,20210603,100u,458681,"Lynn, Lance",1.0,0,1,23.90,9700,CHW,v det,NaN,4,1
4,20210603,151g,656954,"Sheffield, Justus",1.0,0,1,21.35,7900,SEA,@ laa,NaN,6,2


In [25]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [26]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [27]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [28]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [29]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [30]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [31]:
bat_final.to_csv('../merged_data/hitters_6_03.csv')
pitch_final.to_csv('../merged_data/pitchers_6_03.csv')

In [32]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,WAS,6/3/2021,1,607208,Trea Turner,1,Y,SS,L,Tucker Davidson,...,1.0,1,6,3.0,5400,WAS,@ atl,NaN,1,5
1,WAS,6/3/2021,1,543281,Josh Harrison,2,Y,LF,L,Tucker Davidson,...,1.0,2,47,2.0,3100,WAS,@ atl,NaN,1,5
2,WAS,6/3/2021,1,665742,Juan Soto,3,Y,RF,L,Tucker Davidson,...,1.0,3,7,4.0,5000,WAS,@ atl,NaN,1,5
4,WAS,6/3/2021,1,516770,Starlin Castro,5,Y,3B,L,Tucker Davidson,...,1.0,5,5,2.0,3000,WAS,@ atl,NaN,1,5
5,WAS,6/3/2021,1,645302,Victor Robles,6,Y,CF,L,Tucker Davidson,...,1.0,6,7,2.0,3400,WAS,@ atl,NaN,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,SD,6/3/2021,1,592518,Manny Machado,2,N,NaN,R,Taijuan Walker,...,1.0,2,5,4.0,5000,SDG,v nym,NaN,4,3
172,SD,6/3/2021,1,630105,Jake Cronenworth,3,N,NaN,R,Taijuan Walker,...,1.0,3,4,2.0,4400,SDG,v nym,NaN,4,3
174,SD,6/3/2021,1,571976,Wil Myers,6,N,NaN,R,Taijuan Walker,...,1.0,5,7,3.0,4000,SDG,v nym,NaN,4,3
175,SD,6/3/2021,1,595777,Jurickson Profar,7,N,NaN,R,Taijuan Walker,...,1.0,1,7,8.0,3500,SDG,v nym,NaN,4,3


In [33]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,WAS,6/3/2021,1,571578,Patrick Corbin,SP,Y,NaN,1,7,...,1.0,9,1,7.20,6500,WAS,@ atl,NaN,1,5
1,ATL,6/3/2021,1,656353,Tucker Davidson,SP,Y,NaN,2,6,...,1.0,9,1,19.15,4900,ATL,v was,NaN,5,1
2,TB,6/3/2021,1,642232,Ryan Yarbrough,SP,Y,NaN,2,7,...,1.0,0,1,31.15,7300,TAM,@ nyy,NaN,9,2
3,NYY,6/3/2021,1,543037,Gerrit Cole,SP,Y,NaN,3,6,...,1.0,0,1,11.05,11200,NYY,v tam,NaN,2,9
4,BOS,6/3/2021,1,527048,Martin Perez,SP,Y,NaN,1,8,...,1.0,0,1,25.05,7600,BOS,@ hou,NaN,5,1
5,HOU,6/3/2021,1,543606,Jake Odorizzi,SP,Y,NaN,2,7,...,1.0,0,1,0.55,8100,HOU,v bos,NaN,1,5
6,TEX,6/3/2021,1,592314,Mike Foltynewicz,SP,Y,NaN,3,5,...,1.0,9,1,-3.15,5700,TEX,@ col,NaN,6,11
7,MIA,6/3/2021,1,622694,Elieser Hernandez,SP,N,NaN,4,4,...,1.0,9,1,19.45,7600,MIA,@ pit,NaN,3,5
8,PIT,6/3/2021,1,542881,Tyler Anderson,SP,N,NaN,2,6,...,1.0,9,1,11.00,8200,PIT,v mia,NaN,5,3
9,ARI,6/3/2021,1,571676,Seth Frankoff,SP,N,NaN,5,3,...,NaN,9,1,-1.55,5400,ARI,@ mil,NaN,4,7
